In [2]:
import rbot

In [27]:
bb = rbot.DummyBb()
bb.log_load(3)

global_ohlcv=[]


Open:2022 6 21
append324979
Open:2022 6 22
append656022
Open:2022 6 23
append920947


In [53]:
class Agent:
    def __init__(self):
            self.ohlcv = ""

    def on_event(self, time_ms, action, price, size):
        pass
        #//print("E", time_ms, action, price, size)

    def on_tick(self, time_ms, session):
        #print("c", time_ms, market.current_time)
        print("start", rbot.PrintTime(time_ms))        
        # TODO: 出たーがなかったときの処理。
        #session.ohlcv(10, 480)
        # print(market.log_start_ms)
        self.ohlcv = session.ohlcv(60, 100)

        #print(ohlcv)
        return rbot.Order("sell", 1000, 100, 600)

    
    def on_update(self, result):
        print("update", result.status)

In [54]:
agent = Agent()

In [55]:
bb.debug_loop_count = 10

In [56]:
bb.run(agent, 60)

startcall back tick by 60[sec]
call back by update
call back by all log events
 2022-06-21-00:01:00.000
update Open
start 2022-06-21-00:02:00.000
update OpenMake ORDER PyOrder { side: Sell, price: 1000.0, size: 100.0, duration_ms: 600000 }
complete order
Make ORDER PyOrder { side: Sell, price: 1000.0, size: 100.0, duration_ms: 600000 }
complete order

start 2022-06-21-00:03:00.000
update Open
start 2022-06-21-00:04:00.000
Make ORDER PyOrder { side: Sell, price: 1000.0, size: 100.0, duration_ms: 600000 }
complete order
updateMake ORDER PyOrder { side: Sell, price: 1000.0, size: 100.0, duration_ms: 600000 }
complete order
 Open
start 2022-06-21-00:05:00.000
update Open
start 2022-06-21-00:06:00.000
update Open
start 2022-06-21-00:07:00.000
update Open
start 2022-06-21-00:08:00.000
update Open
start 2022-06-21-00:09:00.000
update Open
start 2022-06-21-00:10:00.000
update Open
Make ORDER PyOrder { side: Sell, price: 1000.0, size: 100.0, duration_ms: 600000 }
complete order
Make ORDER PyOrd

In [57]:
tr = bb.transactions

In [58]:
import pandas as pd

In [59]:
for item in tr:
    print(item)

In [60]:
timestamp = []
order_id = []
order_sub_id = []
order_type = []
post_only = []
create_time = []
status = []
open_price = []
close_price = []
size = []
volume = []
profit = []
fee = []
total_profit = []

for item in tr:
    timestamp.append(item.timestamp)
    order_id.append(item.order_id)
    order_sub_id.append(item.order_sub_id)
    order_type.append(item.order_type)
    post_only.append(item.post_only)
    create_time.append(item.create_time)
    status.append(item.status)
    open_price.append(item.open_price)
    close_price.append(item.close_price)
    size.append(item.size)
    volume.append(item.volume)
    profit.append(item.profit)
    fee.append(item.fee)
    total_profit.append(item.total_profit)



In [70]:
df = pd.DataFrame(
data = {"timestamp": timestamp, "order_id": order_id, "order_sub_id": order_sub_id, 
"order_type": order_type, "post_only": post_only, "create_time": create_time,
"status":  status, "open_price": open_price, "close_price": close_price, 
"size": size, "volume": volume, "profit": profit, "fee": fee,
"total_profit":total_profit},
columns = ["timestamp", "order_id", "order_sub_id", "order_type", "post_only",
"create_time", "status", "open_price", "close_price","size", "volume", 
"profit", "fee","total_profit"]
)
df["timestamp"] = pd.to_datetime((df["timestamp"]), utc=True, unit='ms')
df["create_time"] = pd.to_datetime((df["create_time"]), utc=True, unit='ms')


In [71]:
df

,timestamp,order_id,order_sub_id,order_type,post_only,create_time,status,open_price,close_price,size,volume,profit,fee,total_profit
0,2022-06-21 00:01:03+00:00,000000000000-001,0,Sell,True,2022-06-21 00:00:59+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
1,2022-06-21 00:02:01+00:00,000000000000-002,0,Sell,True,2022-06-21 00:01:56+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
2,2022-06-21 00:03:01+00:00,000000000000-003,0,Sell,True,2022-06-21 00:02:59+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
3,2022-06-21 00:04:03+00:00,000000000000-004,0,Sell,True,2022-06-21 00:03:58+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
4,2022-06-21 00:05:01+00:00,000000000000-005,0,Sell,True,2022-06-21 00:04:54+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
5,2022-06-21 00:06:02+00:00,000000000000-006,0,Sell,True,2022-06-21 00:05:59+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
6,2022-06-21 00:07:01+00:00,000000000000-007,0,Sell,True,2022-06-21 00:06:52+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
7,2022-06-21 00:08:00+00:00,000000000000-008,0,Sell,True,2022-06-21 00:07:58+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
8,2022-06-21 00:09:04+00:00,000000000000-009,0,Sell,True,2022-06-21 00:08:59+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06
9,2022-06-21 00:10:00+00:00,000000000000-010,0,Sell,True,2022-06-21 00:09:59+00:00,Open,1000.0,0.0,100.0,0.1,0.0,0.06,-0.06


In [63]:
global_ohlcv

[]

In [65]:
agent.ohlcv

array([[1.65577014e+12, 2.05445000e+04, 2.05975000e+04, 2.05390000e+04,
        2.05795000e+04, 5.12195000e+05],
       [1.65577008e+12, 2.05475000e+04, 2.05550000e+04, 2.05420000e+04,
        2.05445000e+04, 1.78799000e+05],
       [1.65577002e+12, 2.05280000e+04, 2.05780000e+04, 2.05220000e+04,
        2.05495000e+04, 1.13515100e+06],
       [1.65576996e+12, 2.05095000e+04, 2.05425000e+04, 2.05095000e+04,
        2.05285000e+04, 6.04639000e+05],
       [1.65576990e+12, 2.05200000e+04, 2.05560000e+04, 2.05085000e+04,
        2.05150000e+04, 5.51194000e+05],
       [1.65576984e+12, 2.05440000e+04, 2.05440000e+04, 2.05050000e+04,
        2.05180000e+04, 5.18361000e+05],
       [1.65576978e+12, 2.05205000e+04, 2.05465000e+04, 2.05055000e+04,
        2.05440000e+04, 1.19615400e+06],
       [1.65576972e+12, 2.05710000e+04, 2.05875000e+04, 2.05210000e+04,
        2.05210000e+04, 1.54496900e+06],
       [1.65576966e+12, 2.05275000e+04, 2.05710000e+04, 2.05225000e+04,
        2.05710000e+04, 

In [76]:
ohlcv_df = pd.DataFrame(agent.ohlcv, columns=["timestamp", "open", "high", "low", "close", "vol"])
ohlcv_df['timestamp'] = pd.to_datetime((df["timestamp"]), utc=True, unit='ms')

In [77]:
ohlcv_df

,timestamp,open,high,low,close,vol
0,2022-06-21 00:01:03+00:00,20544.5,20597.5,20539.0,20579.5,512195.0
1,2022-06-21 00:02:01+00:00,20547.5,20555.0,20542.0,20544.5,178799.0
2,2022-06-21 00:03:01+00:00,20528.0,20578.0,20522.0,20549.5,1135151.0
3,2022-06-21 00:04:03+00:00,20509.5,20542.5,20509.5,20528.5,604639.0
4,2022-06-21 00:05:01+00:00,20520.0,20556.0,20508.5,20515.0,551194.0
5,2022-06-21 00:06:02+00:00,20544.0,20544.0,20505.0,20518.0,518361.0
6,2022-06-21 00:07:01+00:00,20520.5,20546.5,20505.5,20544.0,1196154.0
7,2022-06-21 00:08:00+00:00,20571.0,20587.5,20521.0,20521.0,1544969.0
8,2022-06-21 00:09:04+00:00,20527.5,20571.0,20522.5,20571.0,1259767.0
9,2022-06-21 00:10:00+00:00,20539.5,20554.5,20516.5,20521.0,601709.0
